In [1]:
import numpy as np
import glob
import matplotlib.pyplot as plt
import skimage.transform

In [2]:
def get_filelist(img_loc):
    filelist = glob.glob(img_loc+'*.jpg')
    filelist.sort()
    return filelist

In [3]:
img_loc = '../ISIC2018_Task3_Training_Input/'

filelist = get_filelist(img_loc)

In [4]:
# Load images as arrays

bs = len(filelist)
input_shape = [bs, 224, 224, 3]
input_shape_inception = [bs, 299, 299, 3]

def get_img_arrs(filelist, input_shape):
    X_orig = np.empty(input_shape)
    bs, n_H, n_W, n_C = input_shape
    for i, fname in enumerate(filelist[:bs]):
        img = plt.imread(fname)
        img = skimage.transform.resize(img, (n_H,n_W), mode='constant')
        X_orig[i] = img
        if i%100 == 0 or i == bs-1:
            print('{} files loaded'.format(i+1))
    return X_orig

In [5]:
X_orig = get_img_arrs(filelist, input_shape)
X_orig_inception = get_img_arrs(filelist, input_shape_inception)

/scratch/user/pdwarkanath/.conda/envs/myenv3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


1 files loaded
101 files loaded
201 files loaded
301 files loaded
401 files loaded
501 files loaded
601 files loaded
701 files loaded
801 files loaded
901 files loaded
1001 files loaded
1101 files loaded
1201 files loaded
1301 files loaded
1401 files loaded
1501 files loaded
1601 files loaded
1701 files loaded
1801 files loaded
1901 files loaded
2001 files loaded
2101 files loaded
2201 files loaded
2301 files loaded
2401 files loaded
2501 files loaded
2601 files loaded
2701 files loaded
2801 files loaded
2901 files loaded
3001 files loaded
3101 files loaded
3201 files loaded
3301 files loaded
3401 files loaded
3501 files loaded
3601 files loaded
3701 files loaded
3801 files loaded
3901 files loaded
4001 files loaded
4101 files loaded
4201 files loaded
4301 files loaded
4401 files loaded
4501 files loaded
4601 files loaded
4701 files loaded
4801 files loaded
4901 files loaded
5001 files loaded
5101 files loaded
5201 files loaded
5301 files loaded
5401 files loaded
5501 files loaded
5601

In [6]:
X_orig_file = 'X_orig.npy'
np.save(X_orig_file, X_orig)

X_orig_inception_file = 'X_orig_inception.npy'
np.save(X_orig_inception_file, X_orig_inception)